# Suite minimisante (linéaire)

In [1]:
from module_optim_topo import *

importing NGSolve-6.2.2105-9-g5a835126f


## 1) Formulation primale

On impose la circulation de $H$ sur le bord du domaine, ce qui en l'absence de courant dans le rotor revient à imposer $\langle H \rangle$ moyen. Cette grandeur est reliée à la valeur d'une alimentation externe en courant.
Pour comparer avec la formulation primale, on choisit $\langle H \rangle = \nu_{d}^* \langle B \rangle$, avec $\langle B \rangle = 1T$ et $\nu_{D}^*$ la réluctivité homogénéisée dans l'axe $d$, qui vaut :

$$ \nu_d^* = \frac{\nu_0}{\theta \mu_r + (1-\theta)} $$

Avec $\theta$ la fraction volumique, égale à 0.5.
Dans cette formulation, les effets de bord se font ressentir sur les assemblages série : $J_d$ est fixée et $J_q$ diminue avec le nombre de barres.

In [2]:
mur=100
maxN=100
hmoy = 1/mu0/(0.5*mur + 0.5)
Jd_primal = []; Jq_primal = []; J_primal = []

for i in range(3,maxN,4):
    mesh = meshLamDisk(i, h = 1/maxN)
    ad, aq = solvePrimal_linear(mur,hmoy,mesh)
    Jd_primal.append(compliance(ad,mesh,1/mu_defaut(mur,mesh)))
    Jq_primal.append(compliance(aq,mesh,1/mu_defaut(mur,mesh)))
    J_primal.append(Jd_primal[-1]-Jq_primal[-1])
    print(f'{i} barres - Jd = {np.round(Jd_primal[-1],2)} - Jq = {np.round(Jq_primal[-1])}')

3 barres - Jd = 24752.68 - Jq = 5740.0
7 barres - Jd = 24752.68 - Jq = 3020.0
11 barres - Jd = 24752.69 - Jq = 2262.0
15 barres - Jd = 24752.69 - Jq = 1908.0
19 barres - Jd = 24752.67 - Jq = 1703.0
23 barres - Jd = 24752.68 - Jq = 1569.0
27 barres - Jd = 24752.68 - Jq = 1475.0
31 barres - Jd = 24752.68 - Jq = 1404.0
35 barres - Jd = 24752.67 - Jq = 1350.0
39 barres - Jd = 24752.71 - Jq = 1306.0
43 barres - Jd = 24752.66 - Jq = 1271.0
47 barres - Jd = 24752.72 - Jq = 1240.0
51 barres - Jd = 24752.69 - Jq = 1215.0
55 barres - Jd = 24752.65 - Jq = 1195.0
59 barres - Jd = 24752.71 - Jq = 1174.0
63 barres - Jd = 24752.72 - Jq = 1156.0
67 barres - Jd = 24752.7 - Jq = 1143.0
71 barres - Jd = 24752.67 - Jq = 1131.0
75 barres - Jd = 24752.63 - Jq = 1120.0
79 barres - Jd = 24752.61 - Jq = 1110.0
83 barres - Jd = 24752.74 - Jq = 1096.0
87 barres - Jd = 24752.76 - Jq = 1085.0
91 barres - Jd = 24752.77 - Jq = 1077.0
95 barres - Jd = 24752.76 - Jq = 1070.0
99 barres - Jd = 24752.72 - Jq = 1064.0


##  2) Formulation duale
On impose le flux de $B$ sur le bord du domaine, ce qui revient à imposer $\langle B \rangle$ moyen. Cette grandeur n'est malheureusement pas reliée directement à une alimentation en magnétostatique, c'est pourquoi elle n'est généralement pas utilisée. En magnétodynamique, elle est reliée à l'intégrale de la tension.
Pour comparer avec la formulation primale, on choisit $\langle B \rangle = 1T$.

Dans cette formulation, les effets de bord se font ressentir sur les assemblages parallèle : $J_q$ est fixée et $J_d$ diminue avec le nombre de barres. Ce comportement est celui qu'on observerait pour les flux habituels (flux de courant, de chaleur).

In [ ]:
Jd_dual = []; Jq_dual = []; J_dual = []

for i in range(3,maxN,4):
    mesh = meshLamDisk(i, h = 1/maxN)
    phid, phiq = solveDual_linear(mur,1,mesh)
    Jd_dual.append(compliance(phid,mesh,1/mu_defaut(mur,mesh)))
    Jq_dual.append(compliance(phiq,mesh,1/mu_defaut(mur,mesh)))
    J_dual.append(Jd_dual[-1]-Jq_dual[-1])
    print(f'{i} barres - Jd = {np.round(Jd_dual[-1],2)} - Jq = {np.round(Jq_dual[-1])}')

## 3) Tracés et comparaisons

In [ ]:
Jpar_lamine = np.pi/2 / (0.5*mu0*mur_fer + 0.5*mu0)
Jperp_lamine = np.pi/2 * (0.5/(mu0*mur_fer) + 0.5/mu0)

plt.semilogy(list(range(3,maxN,4)),J1,'-o')
plt.semilogy(list(range(3,maxN,4)),J2,'-o')
plt.semilogy([1,maxN],[Jpar_lamine,Jpar_lamine],'--')
plt.semilogy([1,maxN],[Jperp_lamine,Jperp_lamine],'--')
plt.legend(["$J_{\parallel}$","$J_{\perp}$","$J_{\parallel}$ asymptotique","$J_{\perp}$ théorique"], fontsize=13)
plt.xlabel("Nombre de barres", fontsize=13)
plt.ylabel("Compliance magnétique (J/m)", fontsize=13)
plt.yticks([1e5,1e6])
plt.grid()
plt.savefig("suite_minimisante_lineaire.pdf",format = 'pdf')